<a href="https://colab.research.google.com/github/Mardav15/oaqjp-final-project-emb-ai/blob/main/text_summarization_idea_gen_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate

In [33]:
# Set up LLM
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=GOOGLE_API_KEY)  #gemini-1.5-flash / gemini-1.5-pro

In [34]:
# Define the State
class State(TypedDict):
    messages: Annotated[list, add_messages]
# Initialize the StateGraph
graph = StateGraph(State)

In [35]:
# Define a prompt template for summarizing
summarizer_prompt = PromptTemplate(
    input_variables=["text"],
    template="""Provide a brief summary of the following text in 2 to 3 sentences: "{text}"."""
)
# Tool to summarize text
@tool
def summarizer(text: str):
    """Summarize a given text into 2 to 3 concise sentences."""
    prompt = summarizer_prompt.format(text=text)
    return llm.invoke(prompt)
# Define a prompt template for generating creative ideas
idea_generator_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Generate 3 creative ideas or suggestions related to the topic: "{topic}"."""
)
# Tool to generate creative ideas
@tool
def idea_generator(topic: str):
    """Generate creative ideas or suggestions for a given topic."""
    prompt = idea_generator_prompt.format(topic=topic)
    return llm.invoke(prompt)

In [36]:
# Bind tools to LLM
tools = [summarizer, idea_generator]
llm_with_tools = llm.bind_tools(tools)

In [37]:
# Create tool nodes
tool_node = ToolNode(tools)
graph.add_node("tool_node", tool_node)

In [38]:
# Agent node
def agent_node(state: State) -> State:
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}
graph.add_node("agent_node", agent_node)

In [39]:
graph.add_edge("tool_node", "agent_node")
graph.set_entry_point("agent_node")

In [40]:
app = graph.compile()
graph_syntax = app.get_graph(xray=True).draw_mermaid()
print(graph_syntax)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	tool_node(tool_node)
	agent_node(agent_node)
	__end__(<p>__end__</p>)
	__start__ --> agent_node;
	agent_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [41]:
text=f"""The IT industry is on the cusp of a significant transformation driven by advancements in technologies like Agentic AI. Unlike traditional automation, Agentic AI enables systems to make decisions autonomously, collaborate with human stakeholders, and adapt dynamically to complex environments. This shift is reshaping the landscape of IT services, creating opportunities for innovation while posing challenges to traditional roles.
Opportunities
Agentic AI has the potential to enhance productivity across the IT industry. With its ability to automate repetitive tasks, such as system monitoring, troubleshooting, and data analysis, organizations can focus on innovation and strategic initiatives. Moreover, the integration of Agentic AI into workflows enables personalized solutions, accelerating software development cycles and improving user experiences. It fosters the creation of new roles centered around AI design, ethics, and optimization, thereby generating employment in emerging domains.
Challenges
However, the rise of Agentic AI also raises concerns about job displacement. Roles that rely heavily on routine tasks may become obsolete, leading to a demand for upskilling and reskilling of the workforce. IT professionals must adapt to roles that require higher-order cognitive skills, such as strategic planning, creative problem-solving, and human-AI collaboration."""

In [42]:
# Invoke the application with a test input
new_state = app.invoke({"messages": f"Summarize the following {text} and then generate new ideas."})
new_state

{'messages': [HumanMessage(content='Summarize the following The IT industry is on the cusp of a significant transformation driven by advancements in technologies like Agentic AI. Unlike traditional automation, Agentic AI enables systems to make decisions autonomously, collaborate with human stakeholders, and adapt dynamically to complex environments. This shift is reshaping the landscape of IT services, creating opportunities for innovation while posing challenges to traditional roles.\nOpportunities\nAgentic AI has the potential to enhance productivity across the IT industry. With its ability to automate repetitive tasks, such as system monitoring, troubleshooting, and data analysis, organizations can focus on innovation and strategic initiatives. Moreover, the integration of Agentic AI into workflows enables personalized solutions, accelerating software development cycles and improving user experiences. It fosters the creation of new roles centered around AI design, ethics, and optim

In [47]:
from langchain_core.messages import AIMessage

# Print the final response
# Check if the last message is an AIMessage and contains tool calls
if isinstance(new_state["messages"][-1], AIMessage) and new_state["messages"][-1].tool_calls:
    # Assuming the first tool call is the summarizer and it has 'text' in args
    summarized_text = new_state["messages"][-1].tool_calls[0]['args']['text']
    print(summarized_text)
elif isinstance(new_state["messages"][-1], AIMessage):
    # If there are no tool calls, print the content of the AIMessage
    print("Agent's Response:")
    print(new_state["messages"][-1].content)
else:
    # Handle other types of messages if necessary
    print("Last message type:", type(new_state["messages"][-1]))
    print(new_state["messages"][-1])

The IT industry is undergoing a significant transformation due to Agentic AI, which enables autonomous decision-making and dynamic adaptation. This shift enhances productivity by automating tasks and enabling personalized solutions, but also raises concerns about job displacement, requiring workforce upskilling and adaptation to new roles.
